# 분류 분석

## 이진 고객 이탈

고객의 웹사이트 광고를 제공하는 마케팅 대행사에서 당사의 고객이탈율이 꽤 높다는 사실을 알게됐습니다.<br>
그 회사에서는 즉시 고객 관리자들을 할당했지만, 어떤 고객이 이탈 할 것인지 예측하는 기계학습 모델을 만들어서 가장 이탈확률이 높은 고객에게 우선적으로 고객 관리자를 배치할 수 있기를 원합니다.<br>
고객이 잠재적 이탈 고객인지 여부를 분류하는 분류모델을 만드세요.

데이터는 customer_churn.csv로 저장돼있습니다. 아래는 각 컬럼의 정의입니다.
- Name : 회사의 최근 담당자의 이름
- Age : 고객의 나이
- Total_Purchase : 구매한 총 광고
- Account_Manager : 바이너리 0 = 고객 관리자 없음, 1 = 고객 관리자 할당됨
- Years : 거래 유지 연도수
- Num_sites : 서비스를 이용 중인 웹사이트 수
- Onboard_date : 마지막 연락처가 등록된 날짜
- Location : 고객 사무실 주소
- Company : 고객 회사의 이름

종속변수

- Churn : 1이면 이탈, 0이면 이탈하지 않음

모델 생성 후에는 고객이 제공하는 new_customers.csv 파일에 저장된 새로운 데이터에 대한 예측값을 제시하세요. <br>
고객은 이 데이터를 통해 관리가 필요한 고객들을 알고 싶어 합니다.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score

### 데이터 불러오기

In [ ]:
df = pd.read_csv("./customer_churn.csv")
df.head()

### 데이터 확인

In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
df.info()

In [ ]:
df.describe(include="all")

1. 20대부터 60대 까지 다양한 연령 분포함
2. 반 이상의 고객들이 고객 관리자가 할당되지 않음
3. 평균 5.27년으로 75% 이상의 사람들이 4년 이상 장기 계약을 유지함
4. 900명 중 약 16%의 사람들이 서비스를 이탈함

In [ ]:
# pairplot 시각화
df_label = ["Age", "Total_Purchase", "Account_Manager", "Years", "Num_Sites", "Churn"]

sns.pairplot(df[df_label])

In [ ]:
# 히트맵 시각화
df_heat = df[["Age", "Total_Purchase", "Account_Manager", "Years", "Num_Sites", "Churn"]]
df_heat = df_heat.columns
df_heat = df[df_heat].corr()
df_heat

In [ ]:
sns.heatmap(df_heat.values, cbar = True, annot= True, fmt = ".3f",
            yticklabels= df_heat, xticklabels= df_heat)
plt.show()

- "Churn" 피처와 관련해서
  - "Num_Sites"와 가장 높은 관련성을 가짐   
    "Num_Sites" > "Years" > "Age" > "Account_Manager" > "Total_Purchase"
  - 대부분의 칼럼과 상관 관계가 낮은 것을 보아 단일 변수로는 이탈여부를 설명 할 수 없음

In [ ]:
df = df[["Age", "Total_Purchase", "Account_Manager", "Years", "Num_Sites", "Churn"]]
df.head()

### 머신러닝용 데이터 전처리

우리는 수치형 데이터만 이용할 예정입니다. Account_Manager는 다루기 어렵지 않기 때문에 모델학습에 포함할 수도 있겠지만 무작위로 할당된 것이기 때문에 큰 의미가 없을 수 있습니다.

In [ ]:
# 결과 받는 df
new_df = pd.read_csv("./new_customers.csv", index_col=False) # index_col = False 를 이용해 모든 열을 데이터 열로 인식
new_df

In [ ]:
new_df.info()

- 결측치 없으므로 바로 진행

#### 1. "Account_Manager"을 사용한 학습

In [ ]:
# 원핫 인코딩
onehot_df = pd.get_dummies(df, columns=["Account_Manager"]).astype(int)
onehot_df = onehot_df[["Age", "Total_Purchase", "Years", "Num_Sites", "Account_Manager_0", "Account_Manager_1", "Churn"]]
onehot_df

In [ ]:
new_df = new_df[["Age", "Total_Purchase", "Account_Manager", "Years", "Num_Sites"]]
onehot_new = pd.get_dummies(new_df, columns=["Account_Manager"]).astype(int)
onehot_new = onehot_new[["Age", "Total_Purchase", "Years", "Num_Sites", "Account_Manager_0", "Account_Manager_1"]]
onehot_new

In [ ]:
# 데이터 분할
x = onehot_df.drop("Churn", axis =1)
y = onehot_df["Churn"]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, stratify=y, random_state=22)

In [ ]:
x_train.shape, x_test.shape

KNN 이진분류

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(onehot_df[["Age", "Total_Purchase", "Years", "Num_Sites", "Account_Manager_0", "Account_Manager_1"]], onehot_df["Churn"])

#### 2. "Account_Manager"을 사용하지 않은 학습

In [ ]:
# df = df.drop("Account_Manager", axis=1)
# df.head() 

#### 3. 기타 : "Onborded_data" 중 년도만 뽑아서 학습하기